<a href="https://colab.research.google.com/github/kumar4372/sentiment_analysis_hands_on/blob/master/(trainer)_Using_RNN_for_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Sentiment Analysis Using Recurrent Neural Network**

---



## choose Hardware accelarator to "GPU" for faster computation. Go to "Runtime" -> "Change runtime type" to change it.


In this tutorial, we will use RNN for sentiment analysis task on movie review dataset.

**What is sentiment analysis?**

Sentiment Analysis is nothing but finding the sentiments of reviews whether it is positive or negative review.

**Example Code to refer**: https://slundberg.github.io/shap/notebooks/deep_explainer/Keras%20LSTM%20for%20IMDB%20Sentiment%20Classification.html

**Notes**
- RNNs are tricky. Choice of batch size is important,
choice of loss and optimizer is critical, etc.
Some configurations won't converge.
- LSTM loss decrease patterns during training can be quite different
from what you see with CNNs/MLPs/etc.

**Importing Libraries**

We start by importing the required dependencies to preprocess our data and build our model.

In [1]:
# Import the dependencies
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers.embeddings import Embedding
from keras.layers import Dense, SimpleRNN,LSTM, GRU
from keras.preprocessing import sequence

print("Imported dependencies.")


Imported dependencies.


**Loading Data**

We will use IMDB sentiment classification dataset which consists of 50,000 movie reviews from IMDB users that are labeled as either positive (1) or negative (0). 

Continue downloading the IMDB dataset, which is, fortunately, already built into Keras.

In [2]:
vocab_size = 10000

# Define the training and test dataset
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=vocab_size)  # vocab_size is no. of words to consider from the dataset, ordering based on frequency.

print("Created test and training data.")
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/usr/local/lib/python3.7/dist-packages/keras/datasets/imdb.py:155: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_train, y_train = np.array(xs[:idx]), np.array(labels[:idx])


Created test and training data.
25000 train sequences
25000 test sequences


/usr/local/lib/python3.7/dist-packages/keras/datasets/imdb.py:156: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_test, y_test = np.array(xs[idx:]), np.array(labels[idx:])


**Exploring the data**

You can see in the output above that the dataset is labeled into two categories, — 0 or 1, which represents the sentiment of the review. The whole dataset contains 9,998 unique words and the average review length is 234 words, with a standard deviation of 173 words.

In [3]:
import numpy as np

#concatenate whole data
data = np.concatenate((x_train, x_test), axis=0)
targets = np.concatenate((y_train, y_test), axis=0)

print("Categories:", np.unique(targets))
print("Number of unique words:", len(np.unique(np.hstack(data))))
length = [len(i) for i in data]
print("Average Review length:", np.mean(length))
print("Standard Deviation:", round(np.std(length)))

Categories: [0 1]
Number of unique words: 9998
Average Review length: 234.75892
Standard Deviation: 173


You can see the first review of the dataset, which is labeled as positive (1). 

In [4]:
print('---review---')
print(x_train[0])
print('---label---')
print(y_train[0])

---review---
[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32]
---label---
1


Above you can see the first review of the dataset, which is labeled as positive (1). The code below retrieves the dictionary mapping word indices back into the original words so that we can read them. It replaces every unknown word with a “#”. It does this by using the get_word_index() function.

In [5]:
index = imdb.get_word_index()
train_text = []
test_text = []
reverse_index = dict([(value, key) for (key, value) in index.items()]) 
for i in range(0,len(x_train)):
  train_text.append(" ".join( [reverse_index.get(i - 3, "#") for i in x_train[i]] ))
for i in range(0,len(x_test)):
  test_text.append(" ".join( [reverse_index.get(i - 3, "#") for i in x_train[i]] ))
print(len(train_text),len(test_text))

25000 25000


In [6]:
print(train_text[0])

# this film was just brilliant casting location scenery story direction everyone's really suited the part they played and you could just imagine being there robert # is an amazing actor and now the same being director # father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for # and would recommend it to everyone to watch and the fly fishing was amazing really cried at the end it was so sad and you know what they say if you cry at a film it must have been good and this definitely was also # to the two little boy's that played the # of norman and paul they were just brilliant children are often left out of the # list i think because the stars that play them all grown up are such a big profile for the whole film but these children are amazing and should be praised for what they have done don't you thi

**Data Preparation**

Now it's time to prepare our data. 

As we know, each review consists of different number of words. Some reviews could even be one word long. e.g. "nice"

Deep learning models look best when all of the data is in a similar shape. 

Here we consider maximum length of our input sequence to be 100. pad_sequences will add 0's to any reviews which don't have a length of 100.

For example, our one word review above would become: "index(nice) 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0... 99 times"

The same goes for any reviews longer than 100 characters, they will be shortened to a maximum of 100.

In [7]:
max_review_length = 100
x_train = sequence.pad_sequences(x_train, maxlen=max_review_length)
x_test = sequence.pad_sequences(x_test, maxlen=max_review_length)

In [8]:
print(x_train[0])

[1415   33    6   22   12  215   28   77   52    5   14  407   16   82
    2    8    4  107  117 5952   15  256    4    2    7 3766    5  723
   36   71   43  530  476   26  400  317   46    7    4    2 1029   13
  104   88    4  381   15  297   98   32 2071   56   26  141    6  194
 7486   18    4  226   22   21  134  476   26  480    5  144   30 5535
   18   51   36   28  224   92   25  104    4  226   65   16   38 1334
   88   12   16  283    5   16 4472  113  103   32   15   16 5345   19
  178   32]


**BUILDING AND TRAINING THE MODEL**

Now our data is ready for some modelling!

Deep learning models have layers.

The top layer takes in the data we've just prepared, the middle layers do some math on this data and the final layer produces an output we can hopefully make use of.

In our case, our model has three layers, 

1. Embedding layer
2. LSTM layer
3. Dense layer.

Our model begins with the line model = Sequential(). Think of this as simply stating "our model will flow from input to output layer in a sequential manner" or "our model goes one step at a time".

**Embedding layer**

The Embedding layer creates a database of the relationships between words.

model.add(Embedding(max_words, embedding_vector_length, input_length=max_review_length)) is saying: add an Embedding layer to our model and use it to turn each of our words into 32 dimensional vector which have some mathematical relationship to each other.

So each of our words will become vectors of dimension 32.

For example, vector of "the" = [0.556433, 0.223122, 0.789654....].

Don't worry for now how this is computed, Keras does it for us.

**LSTM layer**

model.add(LSTM(128)) is saying: add a LSTM layer after our embedding layer in our model and give it 128 units.

LSTM = Long short-term memory. Think of LSTM's as a tap, a tap whichs decides which words flow through the model and which words don't. This layer uses 100 taps to decide which words matter the most in each review.

**Dense layer**

model.add(Dense(1, activation='sigmoid')) is saying: add a Dense layer to the end of our model and use a sigmoid activation function to produce a meaningful output.

A dense layer is also known as a fully-connected layer. This layer connects the 128 LSTM units in the previous layer to 1 unit. This last unit them takes all this information and runs it through a sigmoid function.

Essentially, the sigmoid function will decide if the information should be given a 1 or a -1. 1 for positive and -1 for negative. This is will decided on based on the information passed through by the LSTM layer.


Lastly, we let Keras print a summary of the model we have just built.

In [9]:
# Define how long the embedding vector will be
embedding_vector_length = 128

# Define the layers in the model
model = Sequential()
model.add(Embedding(vocab_size, embedding_vector_length, input_length=vocab_size))
model.add(LSTM(128))
model.add(Dense(1, activation='sigmoid'))

print("Model created.")

Model created.


**Compiling the model**

Now we compile our model, which is nothing but configuring the model for training. We use the “adam” optimizer, an algorithm that changes the weights and biases during training. We also choose binary-crossentropy as loss (because we deal with binary classification) and accuracy as our evaluation metric.

In [10]:
model.compile(loss = 'binary_crossentropy', optimizer='adam',metrics = ['accuracy'])
print("Model compiled, ready to be fit to the training data.")

Model compiled, ready to be fit to the training data.


**Summarize the model**

Making a summary of the model will give us an idea of what's happening at each layer.

In the embedding layer, each of our words is being turned into a vector of dimension 128. Because there are 10000 words (max_words), there are 1,280,000 parameters (128 x 10000).

Parameters are individual pieces of information. The goal of the model is to take a large number of parameters and reduce them down to something we can understand and make use of (less parameters).

The LSTM layer reduces the number of parameters to 82432 = 4 × [128(128+32) + 128].

The final dense layer connects each of the outputs of the LSTM units into one cell (128 + 1).

In [11]:
# Summarize the different layers in the model
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 10000, 128)        1280000   
_________________________________________________________________
lstm (LSTM)                  (None, 128)               131584    
_________________________________________________________________
dense (Dense)                (None, 1)                 129       
Total params: 1,411,713
Trainable params: 1,411,713
Non-trainable params: 0
_________________________________________________________________
None


**Fitting the model to the training data**

Now our model is compiled, it's ready to be set loose on our training data.

We'll be training for 3 epochs with a batch_size of 64.

Because of our loss and optimzation functions, the model accuracy should improve after each cycle.

model.fit(X_train, y_train, epochs=3, batch_size=64) is saying: fit the model we've built on the training dataset for 3 cycles and go over 64 reviews at a time.

Feel free to change the number of epochs (more cycles) or batch_size (more or less information each step) to see how the accuracy changes.

This will take a few minutes.

In [12]:
# Fit the model to the training data
results = model.fit(x_train, y_train, epochs=3, batch_size=64,validation_data=(x_test, y_test))

Epoch 1/3
391/391 [==============================] - 28s 29ms/step - loss: 0.5454 - accuracy: 0.6995 - val_loss: 0.3688 - val_accuracy: 0.8394
Epoch 2/3
391/391 [==============================] - 11s 29ms/step - loss: 0.2623 - accuracy: 0.8949 - val_loss: 0.3545 - val_accuracy: 0.8447
Epoch 3/3
391/391 [==============================] - 12s 29ms/step - loss: 0.1846 - accuracy: 0.9326 - val_loss: 0.3901 - val_accuracy: 0.8388


It is time to evaluate our model:

In [13]:
score, acc = model.evaluate(x_test, y_test,
                            batch_size=64)
print('Test score:', score)
print('Test accuracy:', acc)

391/391 [==============================] - 2s 5ms/step - loss: 0.3901 - accuracy: 0.8388
Test score: 0.39011549949645996
Test accuracy: 0.8388000130653381


Let's analyze the results now and look at some examples.

In [14]:
y_pred = model.predict_classes(x_test)

/usr/local/lib/python3.7/dist-packages/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [15]:
y_pred

array([[0],
       [1],
       [1],
       ...,
       [0],
       [0],
       [0]], dtype=int32)

In [16]:
y_pred.reshape([len(y_pred)])

array([0, 1, 1, ..., 0, 0, 0], dtype=int32)

In [17]:
import pandas as pd
pd.set_option('display.max_colwidth', None) # for better printing

In [18]:
results = pd.DataFrame({"review":test_text, "ground_truth":y_test, "prediction":y_pred.reshape([len(y_pred)])})

In [19]:
results.head(5)

,review,ground_truth,prediction
0,# this film was just brilliant casting location scenery story direction everyone's really suited the part they played and you could just imagine being there robert # is an amazing actor and now the same being director # father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for # and would recommend it to everyone to watch and the fly fishing was amazing really cried at the end it was so sad and you know what they say if you cry at a film it must have been good and this definitely was also # to the two little boy's that played the # of norman and paul they were just brilliant children are often left out of the # list i think because the stars that play them all grown up are such a big profile for the whole film but these children are amazing and should be praised for what they have done don't you think the whole story was so lovely because it was true and was someone's life after all that was shared with us all,0,0
1,# big hair big boobs bad music and a giant safety pin these are the words to best describe this terrible movie i love cheesy horror movies and i've seen hundreds but this had got to be on of the worst ever made the plot is paper thin and ridiculous the acting is an abomination the script is completely laughable the best is the end showdown with the cop and how he worked out who the killer is it's just so damn terribly written the clothes are sickening and funny in equal # the hair is big lots of boobs # men wear those cut # shirts that show off their # sickening that men actually wore them and the music is just # trash that plays over and over again in almost every scene there is trashy music boobs and # taking away bodies and the gym still doesn't close for # all joking aside this is a truly bad film whose only charm is to look back on the disaster that was the 80's and have a good old laugh at how bad everything was back then,1,1
2,# this has to be one of the worst films of the 1990s when my friends i were watching this film being the target audience it was aimed at we just sat watched the first half an hour with our jaws touching the floor at how bad it really was the rest of the time everyone else in the theatre just started talking to each other leaving or generally crying into their popcorn that they actually paid money they had # working to watch this feeble excuse for a film it must have looked like a great idea on paper but on film it looks like no one in the film has a clue what is going on crap acting crap costumes i can't get across how # this is to watch save yourself an hour a bit of your life,1,1
3,# the # # at storytelling the traditional sort many years after the event i can still see in my # eye an elderly lady my friend's mother retelling the battle of # she makes the characters come alive her passion is that of an eye witness one to the events on the # heath a mile or so from where she lives br br of course it happened many years before she was born but you wouldn't guess from the way she tells it the same story is told in bars the length and # of scotland as i discussed it with a friend one night in # a local cut in to give his version the discussion continued to closing time br br stories passed down like this become part of our being who doesn't remember the stories our parents told us when we were children they become our invisible world and as we grow older they maybe still serve as inspiration or as an emotional # fact and fiction blend with # role models warning stories # magic and mystery br br my name is # like my grandfather and his grandfather before him our protagonist introduces himself to us and also introduces the story that stretches back through generations it produces stories within stories stories that evoke the # wonder of scotland its rugged mountains # i

## some samples of wrong predictions

In [20]:
results[results['ground_truth'] != results['prediction']].sample(5)

,review,ground_truth,prediction
1627,# i'm trying to decide if jumping into a wood # would be more enjoyable than this dreck it finishes the destruction of what was once a classic couple of films with jedi menace # and # we have the death of # career he wants us to swallow the # is vader nonsense i never believed it was true this film # those feelings the story hasn't worked since phantom moron and each new film just # the crap on until all that was left was a toy parade i have to go i know where some new rocks to throw are you want spoilers here they come luke and leia are not related vader is not their father duke # should have switched sides while he still could # has less verbal skills than # # his advice has never been any good to anybody # wan # to luke for the first two films # didn't build # he found him in the desert and # to his mom about putting him together from scratch # has # this whole mess with vader and the fall of the republic can be blamed on that stupid b h # # # # if she had any brains she wouldn't have come within a light year of annie but she had told do what george lucas wrote for her what a #,1,0
6549,# dark wolf quick review let's get right to it this is a # piece of rotting # with cow sh t on it it's just an awful movie it's an urban werewolf movie with some of the worst acting imaginable and a story as weak as any # nerd from an 80's high school drama film what's worse is that poor kane # was # into playing the gigantic evil werewolf kane f # # someone's trying to ensure that playing jason # is the height of his film career br br anyway former # # bergman is also in the movie and she eventually becomes a werewolf too it's kind of a crappy cop drama with the world's worst looking werewolf in it but it does have moments of near rampant nudity but that's about all want to know more okay the werewolf is generally an ugly looking black # # around the screen and when we're # enough to actually see a transformation sequence we're presented with something that resembles a full motion video from a video game made during the early stages of the # the first # the cg animation is really that primitive only good for horror hardcore fanatics that want to see small moments of nudity surrounded by rampant visual vomit 2 10 br br www # com,1,0
8330,# well well well as good as john carpenter's season 1 outing in masters of horror was this is the complete opposite he certainly proved he was still a master of horror with cigarette burns but pro life is perhaps the worst i have seen from him br br it's stupid totally devoid of creepy atmosphere and tension and it # it's welcome despite the less than an hour running time the script is nonsense the characters are # and un appealing and the conclusion is beyond absurd br br and for those # who actually bought the dvd one of them being me did you see how carpenter describes the film he's actually proud of it and he talks about it as his best work for a long time and he # the script and in the commentary track where he notices an obvious screw up that made it to the final cut he just says he didn't feel it essential to # the mistake and he just let it be there i fear the old master has completely lost his touch i sincerely hope i'm proved wrong br br i want to leave on a positive note and mention that the creature effects are awesome though technically speaking this film is top notch with effective lighting schemes and make up effects,1,0
13972,# our # of the perverse did good his first time out thats for sure the music is the best you may ever hear by any human but you already know that unless you have no taste or have a brain that is too small to understand greatness a poor script that doesn't flesh out much of a story but at least it has its moments the breathtaking concert stuff is worth seeing it he deserved an oscar for this s t even though he was at times an ego driven # with his # bodyguard chick # always in front a movie that made non fans fans take it or leave it p

## some samples of correct predictions

In [21]:
results[results['ground_truth'] == results['prediction']].sample(5)

,review,ground_truth,prediction
24026,# having just wasted a couple of hours watching this and for 80 of that time in complete disbelief i can give this garbage the turkey of the year award no problem to say the plot was unbelievable is some big understatement frankly i am lost for words to describe this utter tripe not only are the characters completely and utterly without any semblance of originality this sort of stuff has been done much better in dozens of # killer # but the acting was dire for those who pay to see this i hope you get your money back for those who were paid to do this i hope you give your money back believe me folks there are many new releases out there that are much much better go see,1,1
23694,# there is a reason this went straight to video the story is smarmy nick cage plays johnny in a sleazy way sex in # and other scenes that border on tasteless like the scene in the # room # this movie judge # as the # is okay but the movie itself with its themes of # and revenge are not well done but it is a good film for trivia # because so few people saw it,0,0
761,# this movie introduces quite an array of characters and their relationships in the first half hour or so none of them generate any interest or positive response i waited for the intrigue to begin hoping things would get better and ended up sticking around until the bitter end but there was no reward for doing that br br if you want a synopsis look elsewhere to me the action isn't worth # not that the story was that bad i guess you could say i had some problems with the script i e i thought it # a look at the credits will show you that there's a pretty strong cast here used to no # most of the old pros in this flick do good jobs of the actors i hadn't seen much of before i especially liked deborah # # that's about all that i can find good to say about this picture,0,0
5250,# if you like plot turns this is your movie it is impossible at any moment to predict what will happen next nothing is as it appears or ends as you think it will the characters are all gritty and engaging cage is at his best dennis hopper again shows his delightfully sinister side # walsh is perfect in his last performance laura boyle # dwight # makes a film debut superbly in a cameo i # this movie as i am having a really really really bad day film not a slow minute in this film a real sleeper this movie is underrated and sadly overlooked,1,1
17121,# may anne reid and # peter # are paying an apparently # visit to their son bobby steven # and his family in london even as the visit begins # suffers a fatal heart attack leaving may # unsure and questioning her life and future finding herself attracted to her daughter's boyfriend darren daniel craig her actions lead to inevitable consequences br br beautifully filmed but for all its # realism and acclaim the mother offers a collection of mostly unpleasant even # characters and asks the viewer to engage with them reid shines as may and it is her skill and commitment as a wonderfully understated actor that # the film from a completely depressing # but # and # have allowed craig # and # # to create such utterly obnoxious characters that it becomes increasingly difficult to care what happens to may as written the characters played by # and # are in fact so utterly selfish and cold hearted that one begins to wonder what exactly was # trying to say as directed they are either unwilling or unable to lift bobby and paula above the two dimensional in their ghastly # br br worth seeing for # performance but little else a crying shame,0,0


## Below we have some more sample architectures you can try !!

**Extensions**

Let us use LSTM variants. We use check the accuracy by replacing LSTM cell with GRU cell.

In [25]:
# Define the layers in the model
model = Sequential()
model.add(Embedding(vocab_size, embedding_vector_length, input_length=max_review_length))
#model.add(LSTM(128))
model.add(GRU(32,dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))
print(model.summary())

#num_params_layer 3 × [h(h+i) + h]  = 3 × [32(32+64) + 32] = 9312

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 100, 128)          1280000   
_________________________________________________________________
gru_1 (GRU)                  (None, 32)                15552     
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 33        
Total params: 1,295,585
Trainable params: 1,295,585
Non-trainable params: 0
_________________________________________________________________
None


In [26]:
model.compile(loss='binary_crossentropy', 
             optimizer='adam', 
             metrics=['accuracy'])

In [27]:
# Fit the model to the training data
results = model.fit(x_train, y_train, epochs=3, batch_size=64,validation_data=(x_test, y_test))

Epoch 1/3
391/391 [==============================] - 145s 362ms/step - loss: 0.5543 - accuracy: 0.6922 - val_loss: 0.3541 - val_accuracy: 0.8450
Epoch 2/3
391/391 [==============================] - 140s 357ms/step - loss: 0.2739 - accuracy: 0.8919 - val_loss: 0.3519 - val_accuracy: 0.8496
Epoch 3/3
391/391 [==============================] - 140s 358ms/step - loss: 0.1959 - accuracy: 0.9269 - val_loss: 0.3628 - val_accuracy: 0.8500


In [28]:
score, acc = model.evaluate(x_test, y_test,
                            batch_size=64)
print('Test score:', score)
print('Test accuracy:', acc)

391/391 [==============================] - 10s 26ms/step - loss: 0.3628 - accuracy: 0.8500
Test score: 0.36276453733444214
Test accuracy: 0.8499600291252136


**Using LSTM stack layers**

In [29]:
model= Sequential()
model.add(Embedding(vocab_size, embedding_vector_length, input_length=max_review_length))
model.add(LSTM(units=16, return_sequences=True))
model.add(LSTM(units=8, return_sequences=True))
model.add(LSTM(units=4))
model.add(Dense(1, activation='sigmoid'))
print(model.summary())

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 100, 128)          1280000   
_________________________________________________________________
lstm_1 (LSTM)                (None, 100, 16)           9280      
_________________________________________________________________
lstm_2 (LSTM)                (None, 100, 8)            800       
_________________________________________________________________
lstm_3 (LSTM)                (None, 4)                 208       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 5         
Total params: 1,290,293
Trainable params: 1,290,293
Non-trainable params: 0
_________________________________________________________________
None


In [30]:
model.compile(loss='binary_crossentropy', 
             optimizer='adam', 
             metrics=['accuracy'])

In [31]:
# Fit the model to the training data
results = model.fit(x_train, y_train, epochs=3, batch_size=64,validation_data=(x_test, y_test))

Epoch 1/3
391/391 [==============================] - 19s 39ms/step - loss: 0.5557 - accuracy: 0.7192 - val_loss: 0.3775 - val_accuracy: 0.8365
Epoch 2/3
391/391 [==============================] - 14s 37ms/step - loss: 0.2832 - accuracy: 0.8899 - val_loss: 0.3586 - val_accuracy: 0.8409
Epoch 3/3
391/391 [==============================] - 15s 37ms/step - loss: 0.1945 - accuracy: 0.9322 - val_loss: 0.4031 - val_accuracy: 0.8367


In [32]:
score, acc = model.evaluate(x_test, y_test,
                            batch_size=64)
print('Test score:', score)
print('Test accuracy:', acc)

391/391 [==============================] - 3s 8ms/step - loss: 0.4031 - accuracy: 0.8367
Test score: 0.40308964252471924
Test accuracy: 0.8367199897766113


**Using Simple RNN**

In [33]:
# Define the layers in the model
model = Sequential()
model.add(Embedding(vocab_size, embedding_vector_length, input_length=max_review_length))
#model.add(LSTM(128))
model.add(SimpleRNN(128,dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))
print(model.summary())

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 100, 128)          1280000   
_________________________________________________________________
simple_rnn (SimpleRNN)       (None, 128)               32896     
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 129       
Total params: 1,313,025
Trainable params: 1,313,025
Non-trainable params: 0
_________________________________________________________________
None


In [34]:
model.compile(loss='binary_crossentropy', 
             optimizer='adam', 
             metrics=['accuracy'])

In [35]:
# Fit the model to the training data
results = model.fit(x_train, y_train, epochs=3, batch_size=64, validation_data=(x_test, y_test))

Epoch 1/3
391/391 [==============================] - 62s 151ms/step - loss: 0.7025 - accuracy: 0.5154 - val_loss: 0.6812 - val_accuracy: 0.5499
Epoch 2/3
391/391 [==============================] - 59s 150ms/step - loss: 0.6424 - accuracy: 0.6091 - val_loss: 0.7332 - val_accuracy: 0.6783
Epoch 3/3
391/391 [==============================] - 59s 151ms/step - loss: 0.4509 - accuracy: 0.7948 - val_loss: 0.5057 - val_accuracy: 0.7756


In [36]:
score, acc = model.evaluate(x_test, y_test,
                            batch_size=64)
print('Test score:', score)
print('Test accuracy:', acc)

391/391 [==============================] - 4s 9ms/step - loss: 0.5057 - accuracy: 0.7756
Test score: 0.505683958530426
Test accuracy: 0.7756400108337402
